In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
import geopandas as gpd
import seaborn as sns
import scipy.stats as stats

In [3]:
actors = pd.read_csv("./main_data/actors.csv")
countries = pd.read_csv("./main_data/countries.csv")
crew = pd.read_csv("./main_data/crew.csv")
genres = pd.read_csv("./main_data/genres.csv")
languages = pd.read_csv("./main_data/languages.csv")
movies = pd.read_csv("./main_data/movies.csv")
releases = pd.read_csv("./main_data/releases.csv")
rotten_tomatoes_review = pd.read_csv("./main_data/rotten_tomatoes_reviews.csv")
studios = pd.read_csv("./main_data/studios.csv")
the_oscar_awards = pd.read_csv("./main_data/the_oscar_awards.csv")

In [4]:
# Pulizia del dataset actors
nomiNulli = actors["name"].isna().sum()
print(f"Numero di NaN nella colonna 'name' sono: {nomiNulli}")
actors.dropna(subset=["name"],inplace=True)
actors.drop_duplicates(subset=["id", "name", "role"],inplace=True)

Numero di NaN nella colonna 'name' sono: 4


In [5]:
# Pulizia del dataset crew
nomiNulli = crew["name"].isna().sum()
ruoliNulli = crew["role"].isna().sum()

print(f"Numero di NaN nella colonna 'name' sono: {nomiNulli}")
print(f"Numero di NaN nella colonna 'name' sono: {ruoliNulli}")

crew.dropna(subset=["name"],inplace=True)
crew.dropna(subset=["role"],inplace=True)
crew.drop_duplicates(subset=["id", "name", "role"],inplace=True)

Numero di NaN nella colonna 'name' sono: 1
Numero di NaN nella colonna 'name' sono: 0


In [7]:
#pulizia dataset Generi
generiNulli = genres['genre'].isna().sum()
print(f"Numero di NaN nella colonna 'gerne' sono: {nomiNulli}")
genres.dropna(subset=["genre"],inplace=True)
genres.drop_duplicates(subset=["id", "genre"],inplace=True)

Numero di NaN nella colonna 'gerne' sono: 1


In [ ]:
#pulizia dataset lingue
lingueNulle = languages['language'].isna().sum()
TipiNulli = languages['type'].isna().sum()
languages.dropna(subset=["language"],inplace=True)
languages.dropna(subset=["type"],inplace=True)
languages.drop_duplicates(subset=["id", "language", "type"],inplace=True)

In [8]:
# Pulizia del dataset movies
# Eliminiamo gli id duplicati
movies.drop_duplicates(subset=movies.columns.difference(['id']), keep='first',inplace=True)

# Eliminiamo i dulicati che hanno id diverso ma tutti gli altri campi uguali
movies.drop_duplicates(subset=["name","date","tagline","description","minutes","rating"],inplace=True)

# Contiamo, per farci un'idea, i campi NaN
nomiNulli = movies["name"].isna().sum()
print(f"Numero di NaN nella colonna 'name': {nomiNulli}")
dateNulle = movies["date"].isna().sum()
print(f"Numero di NaN nella colonna 'date': {dateNulle}")
minutiNulli = movies["minutes"].isna().sum()
print(f"Numero di NaN nella colonna 'minute': {minutiNulli}")
ratingNullo = movies["rating"].isna().sum()
print(f"Numero di NaN nella colonna 'rating': {ratingNullo}")



# Eliminiamo i 2 dati sui film NaN
movies.dropna(subset=["name"],inplace=True)
nomiNulli = movies["name"].isna().sum()
print(f"Numero di NaN nella colonna 'name': {nomiNulli}")

Numero di NaN nella colonna 'name': 5
Numero di NaN nella colonna 'date': 89827
Numero di NaN nella colonna 'minute': 179090
Numero di NaN nella colonna 'rating': 846551
Numero di NaN nella colonna 'name': 0


In [9]:
non_nan_date_movies = movies["date"].dropna()
# Controlliamo se tutti i valori non NaN sono compresi tra 1888 e 2024
are_all_in_range = non_nan_date_movies.between(1877, 2025).all()

if are_all_in_range:
    print("Tutti i valori non NaN nella colonna 'date' sono compresi tra 1888 e 2024.")
else:
    print("Alcuni valori non NaN nella colonna 'date' NON rientrano nel range 1888-2024.")

#############

# Troviamo i valori non NaN che non sono compresi tra 1888 e 2025
out_of_range = non_nan_date_movies[~non_nan_date_movies.between(1877, 2024)]

print("Valori non NaN che NON rientrano nel range 1888-2024:")
print(out_of_range)

Alcuni valori non NaN nella colonna 'date' NON rientrano nel range 1888-2024.
Valori non NaN che NON rientrano nel range 1888-2024:
3657      2025.0
4679      2026.0
7227      2025.0
7441      2025.0
7787      2025.0
           ...  
932230    2025.0
940009    2030.0
940010    2025.0
940011    2025.0
940012    2025.0
Name: date, Length: 416, dtype: float64


In [10]:
# Pulizia del dataset studios
studioNulli = studios["studio"].isna().sum()
print(f"Numero di NaN nella colonna 'studio': {studioNulli}")

# Eliminiamo gli studios senza nome
studios.dropna(subset=["studio"],inplace=True)

studioNulli = studios["studio"].isna().sum()
print(f"Numero di NaN nella colonna 'studio': {studioNulli}")

Numero di NaN nella colonna 'studio': 10
Numero di NaN nella colonna 'studio': 0


In [11]:
#pulizia del dataset releases
releases.drop_duplicates(subset=["id","country","date","type","rating"])

,id,country,date,type,rating
0,1000001,Andorra,2023-07-21,Theatrical,NaN
1,1000001,Argentina,2023-07-20,Theatrical,ATP
2,1000001,Australia,2023-07-19,Theatrical,PG
3,1000001,Australia,2023-10-01,Digital,PG
4,1000001,Austria,2023-07-20,Theatrical,NaN
...,...,...,...,...,...
1332777,1940967,USA,1909-01-01,Theatrical,NaN
1332778,1940968,Sweden,1908-11-11,Theatrical,NaN
1332779,1940969,France,1902-01-01,Theatrical,NaN
1332780,1940970,France,1902-01-01,Theatrical,NaN
